In [1]:
import os,glob
import cv2
from keras.utils.np_utils import to_categorical
from keras.layers import Activation, Conv2D, Dense, Flatten, MaxPooling2D
from keras.models import Sequential
import numpy as np

Using TensorFlow backend.


In [2]:
input_shape=(32,32,3)
SearchName = ["face","other"]

In [3]:
# 教師データのラベル付け
X_train = [] 
Y_train = [] 
for i in range(len(SearchName)):
    name = SearchName[i]
    # img_file_name_list=os.listdir("../ImageCapture/train_data/"+name)
    # print("Number of {}: {}".format(name, len(img_file_name_list)))
    #for j in range(0,len(img_file_name_list)-1):
   
    in_dir = "../ImageCapture/train_data/"+name+"/*"
    in_jpg=glob.glob(in_dir)
    for j in range(len(in_jpg)):
        file_name = in_jpg[i]
        #n=os.path.join("./FaceEdited/"+SearchName[i]+"/",img_file_name_list[j])  
        img = cv2.imread(file_name)
        if img is None:
            print('image' + str(j) + ':NoImage')
            continue    
        else:
            r,g,b = cv2.split(img)
            img = cv2.merge([r,g,b])
            X_train.append(img)
            Y_train.append(i)
X_train=np.array(X_train)
# X_test=np.array(X_test)
y_train = to_categorical(Y_train)
# y_test = to_categorical(Y_test)

In [4]:
# モデルの定義
model = Sequential()
model.add(Conv2D(input_shape=input_shape, filters=32,kernel_size=(3, 3), 
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(3, 3), 
                 strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(filters=32, kernel_size=(3, 3), 
#                  strides=(1, 1), padding="same"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("sigmoid"))
model.add(Dense(32))
model.add(Activation('sigmoid'))
# 分類したい人数を入れる
model.add(Dense(len(SearchName)))
model.add(Activation('softmax'))

# コンパイル
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
history = model.fit(X_train, y_train, batch_size=70, 
                    epochs=20, verbose=0, validation_data=(X_train, y_train))
#                    epochs=50, verbose=1, validation_data=(X_test, y_test))

In [6]:
# import json
#with open('history.json', 'w') as f:
#    json.dump(history.history, f)
#    json.dump(str(history.history), f)
import pickle
with open('./trainHistoryDict', 'wb') as file_pi:
  pickle.dump(history.history, file_pi)

In [12]:
print(model.summary())
from keras.utils import plot_model
#plot_model(model, to_file='model.png', dpi=64)
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                131136    
_________________________________________________________________
activation_1 (Activation)    (None, 64)               

In [13]:
model.save('my_model.h5')

In [25]:
from keras.models import load_model
model = load_model('./my_model.h5')

In [34]:
v = tf.Variable(0, name='my_variable')
sess = tf.compat.v1.Session()
tf.io.write_graph(sess.graph, './saved_model', 'model.pbtxt')

'./saved_model/model.pbtxt'

In [35]:
pre_model = tf.keras.models.load_model("./my_model.h5")
pre_model.save("saved_model")

INFO:tensorflow:Assets written to: saved_model/assets
